In [37]:
# Bring in dependencies
import os
import asyncio
from typing import List, Literal, Optional
from pydantic import BaseModel, Field

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage, StructuredMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool

from tavily import TavilyClient, MissingAPIKeyError, InvalidAPIKeyError, UsageLimitExceededError, BadRequestError

# Create model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-11-20",
    api_key=os.getenv("OPENAI_API_KEY"),  # Uncomment and set your API key if needed
)

# Initialize Tavily client with proper error handling
def get_tavily_client():
    tavily_api_key = os.getenv("TAVILY_API_KEY")
    # tavily_api_key = ""
    if not tavily_api_key:
        print("Warning: TAVILY_API_KEY environment variable not set. Using mock data.")
        return None
    return TavilyClient(api_key=tavily_api_key)

tavily_client = get_tavily_client()

In [38]:
# Define structured output models for our agents
class CustomerFeedbackAnalysisResult(BaseModel):
    """Result of customer feedback analysis"""
    positive_feedback: List[str] = Field(description="List of positive customer feedback points")
    negative_feedback: List[str] = Field(description="List of negative customer feedback points")
    feature_requests: List[str] = Field(description="List of requested features or improvements")
    sentiment_analysis: str = Field(description="Overall sentiment analysis of customer feedback")
    common_themes: List[str] = Field(description="Common themes in customer feedback")
    reasoning: str = Field(description="Reasoning behind the analysis")

class CompetitorComparisonResult(BaseModel):
    """Result of competitor comparison analysis"""
    competitors: List[str] = Field(description="List of key competitors")
    our_advantages: str = Field(description="Our advantages over competitors")
    our_disadvantages: str = Field(description="Our disadvantages compared to competitors")
    market_position: str = Field(description="Our market position relative to competitors")
    improvement_opportunities: List[str] = Field(description="Opportunities for improvement based on competitor analysis")
    reasoning: str = Field(description="Reasoning behind the analysis")

class CommunityAnalysisResult(BaseModel):
    """Result of community analysis"""
    community_name: str = Field(description="Name of the community analyzed")
    active_topics: List[str] = Field(description="Currently active discussion topics")
    customer_sentiment: str = Field(description="Overall customer sentiment in the community")
    common_questions: List[str] = Field(description="Common questions or issues raised")
    feature_requests: List[str] = Field(description="Feature requests or improvement suggestions")
    reasoning: str = Field(description="Reasoning behind the analysis")

class CompanyResearchReport(BaseModel):
    """Final company research report"""
    customer_feedback: CustomerFeedbackAnalysisResult = Field(description="Customer feedback analysis")
    competitor_comparison: CompetitorComparisonResult = Field(description="Competitor comparison analysis")
    community_analysis: List[CommunityAnalysisResult] = Field(description="Analysis of relevant communities")
    recommendations: List[str] = Field(description="Recommendations based on the research")
    summary: str = Field(description="Executive summary of the research")


In [39]:
# Web Search
async def web_search(query: str, search_depth: Literal["basic", "advanced"]) -> str:
    """
    Search the web for information using Tavily API
    
    Args:
        query: The search query
        search_depth: Either "basic" (faster, 1 credit) or "advanced" (more thorough, 2 credits)
    
    Returns:
        Formatted search results as a string
    """
    if not tavily_client:
        # Enhanced mock data based on the query
        if "persona" in query.lower() or "target audience" in query.lower():
            return f"""Mock search results for: {query}

Summary: Research indicates that IT Directors and CTOs in mid-sized companies (100-1000 employees) are key decision-makers for business technology optimization services. They typically have 10+ years of experience, are aged 35-55, and face challenges with legacy system integration and digital transformation.

Sources:
1. The Modern IT Director: Challenges and Opportunities
   URL: https://example.com/it-director-research
   Snippet: IT Directors in mid-sized companies report spending 60% of their time on maintenance rather than innovation. Key pain points include budget constraints, talent shortages, and keeping up with rapid technological changes...

2. CTO Survey 2023: Technology Decision Making
   URL: https://example.com/cto-survey-2023
   Snippet: 78% of CTOs cite "optimizing existing technology investments" as a top priority. They typically evaluate 3-5 vendors before making significant technology decisions...

3. B2B Technology Buying Behavior
   URL: https://example.com/b2b-tech-buying
   Snippet: Technology purchase decisions in mid-sized companies typically involve 6-10 stakeholders, with the IT Director or CTO having the most influence. The buying cycle averages 3-6 months...
"""
        elif "pain point" in query.lower():
            return f"""Mock search results for: {query}

Summary: IT Directors and CTOs in professional services face several key pain points: 1) Integration challenges between legacy and new systems, 2) Difficulty measuring ROI on technology investments, 3) Talent shortages for specialized tech roles, 4) Security concerns with distributed workforces, and 5) Budget constraints limiting innovation.

Sources:
1. Technology Pain Points in Professional Services
   URL: https://example.com/tech-pain-points
   Snippet: A survey of 500 IT leaders revealed that 72% struggle with integrating legacy systems with modern cloud solutions. This creates data silos that hamper decision-making and operational efficiency...

2. The ROI Challenge: Measuring Technology Impact
   URL: https://example.com/tech-roi-measurement
   Snippet: Only 34% of companies report having clear metrics to measure the business impact of their technology investments. This creates challenges in justifying further spending...

3. IT Talent Gap Analysis 2023
   URL: https://example.com/it-talent-gap
   Snippet: 68% of IT Directors report difficulty hiring and retaining specialized technical talent, particularly in cybersecurity, data science, and cloud architecture...
"""
        elif "competitor" in query.lower():
            return f"""Mock search results for: {query}

Summary: The business technology optimization consulting space has several key players including Accenture, Deloitte Digital, IBM Consulting, Capgemini, and boutique firms like Slalom and West Monroe. Large firms offer comprehensive services but are often expensive and less agile, while boutique firms provide more personalized service but may lack global reach.

Sources:
1. Technology Consulting Market Analysis 2023
   URL: https://example.com/tech-consulting-market
   Snippet: The global technology optimization consulting market is valued at $195 billion and growing at 8.3% annually. Major players control 65% of market share, with mid-sized and boutique firms gaining ground by specializing in specific industries...

2. Consulting Firm Comparison: Enterprise vs. Boutique
   URL: https://example.com/consulting-comparison
   Snippet: While Accenture and Deloitte dominate with comprehensive service offerings, boutique firms like Slalom report higher client satisfaction scores (NPS of 72 vs. 64 for larger firms)...

3. Technology Consulting Pricing Models
   URL: https://example.com/consulting-pricing
   Snippet: Enterprise consulting firms typically charge 30-50% more than mid-sized competitors. However, they offer broader capabilities and greater resources for complex, global implementations...
"""
        else:
            return f"""Mock search results for: {query}

Summary: The search results provide information relevant to {query}, highlighting key trends, challenges, and opportunities in this area.

Sources:
1. Recent Research on {query.title()}
   URL: https://example.com/research-{query.replace(' ', '-')}
   Snippet: Studies show that {query} is becoming increasingly important in the business landscape, with organizations investing more resources in this area...

2. Best Practices for {query.title()}
   URL: https://example.com/best-practices-{query.replace(' ', '-')}
   Snippet: Industry leaders recommend a strategic approach to {query}, focusing on measurable outcomes and stakeholder alignment...

3. Future Trends in {query.title()}
   URL: https://example.com/trends-{query.replace(' ', '-')}
   Snippet: Experts predict significant evolution in {query} over the next 3-5 years, with emerging technologies playing a crucial role...
"""
    
    try:
        # Original Tavily implementation remains unchanged
        # Convert to synchronous call since Tavily doesn't have async API
        response = tavily_client.search(
            query=query,
            search_depth=search_depth,
            include_answer=True,
            max_results=5
        )
        
        results = f"Search results for: {query}\n\n"
        
        # Add the AI-generated answer if available
        if response.get("answer"):
            results += f"Summary: {response.get('answer')}\n\n"
        
        results += "Sources:\n"
        for i, result in enumerate(response.get("results", []), 1):
            title = result.get("title", "No title")
            url = result.get("url", "No URL")
            content_snippet = result.get("content", "")[:200] + "..." if result.get("content") else ""
            
            results += f"{i}. {title}\n"
            results += f"   URL: {url}\n"
            results += f"   Snippet: {content_snippet}\n\n"
        
        return results
    except (MissingAPIKeyError, InvalidAPIKeyError) as e:
        return f"API key error: {str(e)}\nPlease check your Tavily API key configuration."
    except UsageLimitExceededError:
        return f"Tavily API usage limit exceeded. Consider upgrading your plan."
    except Exception as e:
        return f"Error performing search: {str(e)}"


async def find_communities(persona: str, platform: str) -> str:
    """
    Find online communities for a specific persona
    
    Args:
        persona: The target persona/audience
        platform: Optional specific platform to search for communities
    
    Returns:
        List of relevant communities
    """
    if not tavily_client:
        # Fallback to mock data
        communities = f"Communities for {persona}:\n"
        communities += f"- Reddit r/{persona.lower().replace(' ', '')}\n"
        communities += "- LinkedIn groups for professionals\n"
        communities += "- Facebook industry groups\n"
        return communities
    
    try:
        platform_str = f"on {platform}" if platform else ""
        query = f"online communities for {persona} {platform_str} discussion forums groups"
        
        response = tavily_client.search(
            query=query,
            search_depth="basic",
            include_answer=True,
            max_results=5
        )
        
        results = f"Communities for {persona}{' on ' + platform if platform else ''}:\n\n"
        
        # Add the AI-generated answer if available
        if response.get("answer"):
            results += f"{response.get('answer')}\n\n"
        
        results += "Sources:\n"
        for i, result in enumerate(response.get("results", []), 1):
            title = result.get("title", "No title")
            url = result.get("url", "No URL")
            
            results += f"{i}. {title}\n"
            results += f"   URL: {url}\n\n"
        
        return results
    except Exception as e:
        # Fallback to mock data if search fails
        fallback = f"Communities for {persona}:\n"
        fallback += f"- Reddit r/{persona.lower().replace(' ', '')}\n"
        fallback += "- LinkedIn groups for professionals\n"
        fallback += "- Facebook industry groups\n"
        fallback += f"Note: Using fallback data due to search error: {str(e)}"
        return fallback

async def industry_trends(industry: str, timeframe: str) -> str:
    """
    Search for industry trends in a specific timeframe
    
    Args:
        industry: The industry to research
        timeframe: Time period for trends (recent, yearly, etc.)
    
    Returns:
        Formatted trend information
    """
    if not tavily_client:
        return f"Mock {timeframe} trends in {industry} industry:\n- Trend 1: Example trend\n- Trend 2: Another trend"
    
    try:
        query = f"latest trends in {industry} industry {timeframe}"
        response = tavily_client.search(
            query=query,
            search_depth="basic",
            include_answer=True,
            max_results=5
        )
        
        results = f"{timeframe.capitalize()} trends in {industry}:\n\n"
        
        # Add the AI-generated answer if available
        if response.get("answer"):
            results += f"{response.get('answer')}\n\n"
        
        results += "Sources:\n"
        for i, result in enumerate(response.get("results", []), 1):
            title = result.get("title", "No title")
            url = result.get("url", "No URL")
            
            results += f"{i}. {title}\n"
            results += f"   URL: {url}\n\n"
        
        return results
    except Exception as e:
        return f"Error searching for industry trends: {str(e)}"

# Create function tools with the updated implementations
web_search_tool = FunctionTool(web_search, description="Search the web for information", strict=True)
find_communities_tool = FunctionTool(find_communities, description="Find online communities for a specific persona", strict=True)
industry_trends_tool = FunctionTool(industry_trends, description="Research trends in a specific industry", strict=True)

In [40]:
# Mock data for testing
def create_mock_company_research_report():
    """Create a mock company research report for testing purposes"""
    feedback = CustomerFeedbackAnalysisResult(
        positive_feedback=[
            "Excellent customer service response time",
            "Intuitive user interface",
            "Reliable product performance",
            "Good value for money",
            "Regular feature updates"
        ],
        negative_feedback=[
            "Documentation could be more comprehensive",
            "Some features are difficult to discover",
            "Mobile app needs improvement",
            "Pricing structure is complex",
            "Integration with third-party tools is limited"
        ],
        feature_requests=[
            "Better mobile experience",
            "More integration options",
            "Enhanced reporting capabilities",
            "Customizable dashboard",
            "API access for enterprise users"
        ],
        sentiment_analysis="Overall positive sentiment with 75% positive feedback, 15% neutral, and 10% negative",
        common_themes=[
            "Ease of use",
            "Customer support quality",
            "Feature requests",
            "Integration needs",
            "Pricing concerns"
        ],
        reasoning="Analysis based on customer reviews, support tickets, and social media mentions"
    )
    
    competitors = CompetitorComparisonResult(
        competitors=["Competitor A", "Competitor B", "Competitor C"],
        our_advantages="Superior customer support, more intuitive interface, better pricing for small businesses",
        our_disadvantages="Fewer integration options, less comprehensive documentation, limited enterprise features",
        market_position="Strong position in small to medium business segment, growing presence in enterprise market",
        improvement_opportunities=[
            "Expand integration capabilities",
            "Enhance documentation and training materials",
            "Develop enterprise-grade features",
            "Simplify pricing structure",
            "Improve mobile experience"
        ],
        reasoning="Based on feature comparison, customer reviews, and market analysis"
    )
    
    communities = [
        CommunityAnalysisResult(
            community_name="Reddit r/ProductUsers",
            active_topics=[
                "Feature requests and suggestions",
                "Troubleshooting common issues",
                "Best practices and tips",
                "Integration discussions",
                "Pricing and value discussions"
            ],
            customer_sentiment="Generally positive with active user engagement",
            common_questions=[
                "How to integrate with X?",
                "Best practices for Y?",
                "Pricing structure explanation",
                "Mobile app features",
                "Enterprise features availability"
            ],
            feature_requests=[
                "Mobile app improvements",
                "More integration options",
                "Enhanced reporting",
                "Custom dashboards",
                "API access"
            ],
            reasoning="Analysis of community discussions and engagement patterns"
        )
    ]
    
    report = CompanyResearchReport(
        customer_feedback=feedback,
        competitor_comparison=competitors,
        community_analysis=communities,
        recommendations=[
            "Prioritize mobile app improvements based on user feedback",
            "Develop comprehensive documentation and training materials",
            "Expand integration capabilities with popular third-party tools",
            "Simplify pricing structure for better transparency",
            "Enhance enterprise features to compete in larger market"
        ],
        summary="This research report analyzes customer feedback, competitor positioning, and community discussions to provide actionable insights for product improvement and market positioning."
    )
    
    return report

# Add a function to test with mock data
def test_with_mock_data():
    """Run a test using mock data instead of the full workflow"""
    report = create_mock_company_research_report()
    
    # Print the final report in a structured format
    print("\n=== MOCK COMPANY RESEARCH REPORT ===")
    print(f"EXECUTIVE SUMMARY:\n{report.summary}\n")
    
    print("CUSTOMER FEEDBACK:")
    print("Positive Feedback:")
    for feedback in report.customer_feedback.positive_feedback[:3]:
        print(f"- {feedback}")
    
    print("\nNegative Feedback:")
    for feedback in report.customer_feedback.negative_feedback[:3]:
        print(f"- {feedback}")
    
    print("\nFeature Requests:")
    for request in report.customer_feedback.feature_requests[:3]:
        print(f"- {request}")
    
    print("\nCOMPETITOR ANALYSIS:")
    for comp in report.competitor_comparison.competitors:
        print(f"- {comp}")
    
    print("\nOur Advantages:")
    print(report.competitor_comparison.our_advantages)
    
    print("\nOur Disadvantages:")
    print(report.competitor_comparison.our_disadvantages)
    
    print("\nCOMMUNITY ANALYSIS:")
    for community in report.community_analysis:
        print(f"\nCommunity: {community.community_name}")
        print("Active Topics:")
        for topic in community.active_topics[:3]:
            print(f"- {topic}")
    
    print("\nRECOMMENDATIONS:")
    for i, rec in enumerate(report.recommendations, 1):
        print(f"{i}. {rec}")
    
    return report

In [41]:
# Create the agents
company_research_manager = UserProxyAgent(
    name="Company_Research_Manager",
    description="You are a company research manager responsible for coordinating research tasks and presenting findings. You'll work with specialized agents to conduct comprehensive company research.",
)

feedback_analyzer = AssistantAgent(
    name="Feedback_Analyzer",
    model_client=model_client,
    tools=[web_search_tool],
    system_message="You are an expert in analyzing customer feedback, reviews, and support tickets. Use web search to find and analyze customer feedback across various platforms.",
    output_content_type=CustomerFeedbackAnalysisResult,
)

competitor_analyzer = AssistantAgent(
    name="Competitor_Analyzer",
    model_client=model_client,
    tools=[web_search_tool],
    system_message="You are an expert in analyzing competitor offerings and market positioning. Compare our company's products/services with competitors to identify strengths, weaknesses, and opportunities.",
    output_content_type=CompetitorComparisonResult,
)

community_analyzer = AssistantAgent(
    name="Community_Analyzer",
    model_client=model_client,
    tools=[web_search_tool, find_communities_tool],
    system_message="You are an expert in analyzing online communities and discussions. Identify relevant communities, analyze discussions, and extract valuable insights about customer needs and feedback.",
    output_content_type=CommunityAnalysisResult,
)

In [42]:
# Perform company research
async def conduct_company_research(company_name: str, product_service: str):
    # Initialize the cancellation token
    cancellation_token = CancellationToken()
    
    print("=== Starting Company Research Process ===")
    
    try:
        # Step 1: Customer Feedback Analysis
        print("\n=== Step 1: Customer Feedback Analysis ===")
        feedback_task = f"Analyze customer feedback for {company_name}'s {product_service}. Focus on reviews, social media mentions, and support tickets."
        feedback_response = await feedback_analyzer.on_messages(
            [TextMessage(content=feedback_task, source=company_research_manager.name)],
            cancellation_token=cancellation_token,
        )
        feedback_result = feedback_response.chat_message.content
        print(f"Customer Feedback Analysis Complete: {len(feedback_result.positive_feedback)} positive points identified")
        
        # Step 2: Competitor Comparison
        print("\n=== Step 2: Competitor Comparison ===")
        competitor_task = f"Compare {company_name}'s {product_service} with competitors. Focus on strengths, weaknesses, and market positioning."
        competitor_response = await competitor_analyzer.on_messages(
            [TextMessage(content=competitor_task, source=company_research_manager.name)],
            cancellation_token=cancellation_token,
        )
        competitor_result = competitor_response.chat_message.content
        print(f"Competitor Comparison Complete: {len(competitor_result.competitors)} competitors analyzed")
        
        # Step 3: Community Analysis
        print("\n=== Step 3: Community Analysis ===")
        community_task = f"Analyze relevant communities discussing {company_name} and {product_service}. Focus on customer discussions, questions, and feedback."
        community_response = await community_analyzer.on_messages(
            [TextMessage(content=community_task, source=company_research_manager.name)],
            cancellation_token=cancellation_token,
        )
        community_result = community_response.chat_message.content
        # Wrap the single result in a list
        community_results = [community_result]
        print(f"Community Analysis Complete: {len(community_results)} communities analyzed")
        
        # Step 4: Generate Final Report
        print("\n=== Step 4: Generating Final Report ===")
        
        final_report = CompanyResearchReport(
            customer_feedback=feedback_result,
            competitor_comparison=competitor_result,
            community_analysis=[community_result],
            recommendations=[
                f"Address top customer pain point: {feedback_result.negative_feedback[0] if feedback_result.negative_feedback else 'No major issues identified'}",
                f"Leverage key advantage: {competitor_result.our_advantages.split('.')[0]}",
                f"Implement most requested feature: {feedback_result.feature_requests[0] if feedback_result.feature_requests else 'No specific feature requests identified'}",
                "Develop targeted content addressing common customer questions",
                "Create improvement plan based on competitor analysis"
            ],
            summary=f"This company research report analyzes {company_name}'s {product_service}, identifying {len(feedback_result.positive_feedback)} positive aspects and {len(feedback_result.negative_feedback)} areas for improvement. The research includes analysis of {len(competitor_result.competitors)} competitors and {len(community_results)} relevant communities, providing actionable insights for product improvement and market positioning."
        )
        
        print("\n=== Company Research Complete ===")
        return final_report
    
    except Exception as e:
        print(f"\n=== Error in Company Research Process: {str(e)} ===")
        print("Falling back to mock data...")
        return create_mock_company_research_report()

In [43]:
# Call the function
COMPANY_NAME = "BluePrint Engines"
PRODUCT_SERVICE = "Crate Engines for enthusiasts"

# Run the research
async def main():
    report = await conduct_company_research(COMPANY_NAME, PRODUCT_SERVICE)
    
    # Print the final report in a structured format
    print("\n=== FINAL MARKET RESEARCH REPORT ===")
    print(f"EXECUTIVE SUMMARY:\n{report.summary}\n")
    
    print("TARGET PERSONA:")
    print(f"Name: {report.target_persona.persona_name}")
    print(f"Demographics: {report.target_persona.demographics}")
    print(f"Behaviors: {report.target_persona.behaviors}")
    print(f"Needs: {report.target_persona.needs}")
    print(f"Online Communities: {', '.join(report.target_persona.online_communities)}")
    
    print("\nKEY PAIN POINTS:")
    for i, pain in enumerate(report.pain_points.prioritization[:3], 1):
        print(f"{i}. {pain}")
    
    print("\nCOMPETITOR ANALYSIS:")
    for comp in report.competitors.competitors[:3]:
        print(f"- {comp}")
        # Since strengths and weaknesses are now strings, not dictionaries
        print(f"  Strengths: {report.competitors.strengths}")
        print(f"  Weaknesses: {report.competitors.weaknesses}")
    
    print("\nRECOMMENDATIONS:")
    for i, rec in enumerate(report.recommendations, 1):
        print(f"{i}. {rec}")
    
    # Close the model client
    await model_client.close()

# For Jupyter notebooks, provide a synchronous version
def run_research_sync(use_mock=False):
    # Import required here to avoid circular imports
    import asyncio
    import nest_asyncio
    import time
    
    # Apply nest_asyncio to allow nested event loops in Jupyter
    nest_asyncio.apply()
    
    # If use_mock is True, just return mock data without running the full workflow
    if use_mock:
        print("Using mock data as requested...")
        return test_with_mock_data()
    
    # Get the current event loop
    loop = asyncio.get_event_loop()
    
    try:
        # Run the conduct_company_research function with the global variables
        return loop.run_until_complete(conduct_company_research(COMPANY_NAME, PRODUCT_SERVICE))
    except RuntimeError as e:
        print(f"Runtime error: {e}")
        print("Trying alternative approach for Jupyter environment...")
        # Alternative approach if the above fails
        from IPython.lib.backgroundjobs import BackgroundJobManager
        jobs = BackgroundJobManager()
        result = [None]
        
        def run_async():
            result[0] = asyncio.run(conduct_company_research(COMPANY_NAME, PRODUCT_SERVICE))
            
        jobs.new(run_async)
        while result[0] is None:
            time.sleep(0.1)
        return result[0]
    except Exception as e:
        print(f"Error: {str(e)}")
        print("Falling back to mock data...")
        return test_with_mock_data()

In [44]:
# Format the report
def format_company_research_report(report):
    """
    Format a CompanyResearchReport object into a well-structured, readable document
    
    Args:
        report: A CompanyResearchReport object containing the research data
        
    Returns:
        A formatted string containing the report in markdown format
    """
    # Create the markdown document
    markdown = "# Company Research Report\n\n"
    
    # Executive Summary section
    markdown += "## Executive Summary\n\n"
    markdown += f"{report.summary}\n\n"
    
    # Customer Feedback section
    markdown += "## Customer Feedback Analysis\n\n"
    
    markdown += "### Positive Feedback\n"
    markdown += "\n".join([f"- {feedback}" for feedback in report.customer_feedback.positive_feedback]) + "\n\n"
    
    markdown += "### Negative Feedback\n"
    markdown += "\n".join([f"- {feedback}" for feedback in report.customer_feedback.negative_feedback]) + "\n\n"
    
    markdown += "### Feature Requests\n"
    markdown += "\n".join([f"- {request}" for request in report.customer_feedback.feature_requests]) + "\n\n"
    
    markdown += "### Sentiment Analysis\n"
    markdown += f"{report.customer_feedback.sentiment_analysis}\n\n"
    
    markdown += "### Common Themes\n"
    markdown += "\n".join([f"- {theme}" for theme in report.customer_feedback.common_themes]) + "\n\n"
    
    # Competitor Analysis section
    markdown += "## Competitor Comparison\n\n"
    
    markdown += "### Key Competitors\n"
    markdown += "\n".join([f"- {competitor}" for competitor in report.competitor_comparison.competitors]) + "\n\n"
    
    markdown += "### Our Advantages\n"
    markdown += f"{report.competitor_comparison.our_advantages}\n\n"
    
    markdown += "### Our Disadvantages\n"
    markdown += f"{report.competitor_comparison.our_disadvantages}\n\n"
    
    markdown += "### Market Position\n"
    markdown += f"{report.competitor_comparison.market_position}\n\n"
    
    markdown += "### Improvement Opportunities\n"
    markdown += "\n".join([f"- {opportunity}" for opportunity in report.competitor_comparison.improvement_opportunities]) + "\n\n"
    
    # Community Analysis section
    markdown += "## Community Analysis\n\n"
    
    for i, community in enumerate(report.community_analysis):
        markdown += f"### {community.community_name}\n\n"
        
        markdown += "#### Active Topics\n"
        markdown += "\n".join([f"- {topic}" for topic in community.active_topics]) + "\n\n"
        
        markdown += "#### Customer Sentiment\n"
        markdown += f"{community.customer_sentiment}\n\n"
        
        markdown += "#### Common Questions\n"
        markdown += "\n".join([f"- {question}" for question in community.common_questions]) + "\n\n"
        
        markdown += "#### Feature Requests\n"
        markdown += "\n".join([f"- {request}" for request in community.feature_requests]) + "\n\n"
    
    # Recommendations section
    markdown += "## Recommendations\n\n"
    markdown += "\n".join([f"{i+1}. {recommendation}" for i, recommendation in enumerate(report.recommendations)]) + "\n\n"
    
    return markdown

# Update the display function to use the correct formatter
def display_formatted_report(report=None):
    if report is None:
        report = create_mock_company_research_report()
    
    formatted_report = format_company_research_report(report)
    
    from IPython.display import Markdown, display
    display(Markdown(formatted_report))
    
    return formatted_report

# Example usage
def display_formatted_report(report=None):
    """
    Display a formatted market research report
    
    Args:
        report: Optional MarketResearchReport object. If None, creates a mock report.
    """
    if report is None:
        report = create_mock_company_research_report()
    
    formatted_report = format_company_research_report(report)
    
    # In a Jupyter notebook, you can display markdown directly
    from IPython.display import Markdown, display
    display(Markdown(formatted_report))
    
    # Alternatively, print the markdown (less pretty but works everywhere)
    # print(formatted_report)
    
    return formatted_report

In [45]:
# To display the report from a previous research run:
report = run_research_sync()
display_formatted_report(report)

# Or to display a mock report for testing:
# display_formatted_report()

# Testing
# run_research_sync(use_mock=True)
# test_with_mock_data()

=== Starting Company Research Process ===

=== Step 1: Customer Feedback Analysis ===
Customer Feedback Analysis Complete: 3 positive points identified

=== Step 2: Competitor Comparison ===
Competitor Comparison Complete: 3 competitors analyzed

=== Step 3: Community Analysis ===
Community Analysis Complete: 1 communities analyzed

=== Step 4: Generating Final Report ===

=== Company Research Complete ===


# Company Research Report

## Executive Summary

This company research report analyzes BluePrint Engines's Crate Engines for enthusiasts, identifying 3 positive aspects and 3 areas for improvement. The research includes analysis of 3 competitors and 1 relevant communities, providing actionable insights for product improvement and market positioning.

## Customer Feedback Analysis

### Positive Feedback
- Customers appreciate the robust performance of BluePrint Engines' products.
- The warranty coverage provided is generally well-received.
- Dyno-tested and manufactured engines are appealing to the audience.

### Negative Feedback
- Some reviews mention issues with delivery timelines and product availability.
- Customer service interactions and response times have been criticized.
- Issues with certain technical support details or clarity on parts specifications.

### Feature Requests
- Improvements in customer service responsiveness.
- More transparent updates on engine delivery schedules.
- Expanded clarification resources for technical queries and installation.

### Sentiment Analysis
Mixed with both positive experiences regarding product quality and negative sentiment on service response and delivery concerns.

### Common Themes
- Performance and warranty coverage are key positives.
- Delivery delays and customer service are recurring negative points.
- Engagement through social media and community feedback is notable.
- Customer service interaction quality has room for improvement.

## Competitor Comparison

### Key Competitors
- GM Performance
- ATK Engines
- Shafiroff Racing Engines

### Our Advantages
BluePrint Engines excels in providing high-quality factory-built crate engines, coupled with reliable warranties and technical support. This makes our offerings accessible and straightforward for automotive enthusiasts.

### Our Disadvantages
Compared to competitors, BluePrint Engines may lack the customization options and exclusivity that more specialized firms offer, which might be preferable to some niche market segments.

### Market Position
BluePrint Engines is well-positioned in the market as a reliable factory-backed provider of crate engines, catering to a wide range of enthusiasts with balanced quality and cost efficiency.

### Improvement Opportunities
- Expand customizable options to target niche segments seeking tailored solutions.
- Enhance perceived exclusivity through premium-level offerings or branding.

## Community Analysis

### r/Mustangs

#### Active Topics
- Difference between crate engines and rebuilt engines
- Comparison of Ford Performance engines to BluePrint engines

#### Customer Sentiment
Generally positive

#### Common Questions
- Compatibility of BluePrint crate engines with specific models
- Power outputs and reliability of crate engines

#### Feature Requests
- Improvement in warranty terms
- Offers on installation services for crate engines

## Recommendations

1. Address top customer pain point: Some reviews mention issues with delivery timelines and product availability.
2. Leverage key advantage: BluePrint Engines excels in providing high-quality factory-built crate engines, coupled with reliable warranties and technical support
3. Implement most requested feature: Improvements in customer service responsiveness.
4. Develop targeted content addressing common customer questions
5. Create improvement plan based on competitor analysis



"# Company Research Report\n\n## Executive Summary\n\nThis company research report analyzes BluePrint Engines's Crate Engines for enthusiasts, identifying 3 positive aspects and 3 areas for improvement. The research includes analysis of 3 competitors and 1 relevant communities, providing actionable insights for product improvement and market positioning.\n\n## Customer Feedback Analysis\n\n### Positive Feedback\n- Customers appreciate the robust performance of BluePrint Engines' products.\n- The warranty coverage provided is generally well-received.\n- Dyno-tested and manufactured engines are appealing to the audience.\n\n### Negative Feedback\n- Some reviews mention issues with delivery timelines and product availability.\n- Customer service interactions and response times have been criticized.\n- Issues with certain technical support details or clarity on parts specifications.\n\n### Feature Requests\n- Improvements in customer service responsiveness.\n- More transparent updates on e